##Analytic Model

In [0]:
---Clients amount per country that finished the process
SELECT estado, COUNT(*) AS total_clientes
  FROM digital_funnel.gold.pv_clientesfunnel
 WHERE proceso_completo = 1
GROUP BY estado
ORDER BY estado;

In [0]:
---Clients amount per country that didn't finis the process
SELECT estado, COUNT(*) AS total_clientes
  FROM digital_funnel.gold.pv_clientesfunnel
 WHERE proceso_completo = 0
GROUP BY estado
ORDER BY estado;

In [0]:
---Clients amount per country that finished the process but has not all the stages (i.e. proceso_completo = 0)
SELECT estado, COUNT(*) AS total_clientes
  FROM digital_funnel.gold.pv_clientesfunnel
 WHERE proceso_completo = 0 AND confirmacion_de_datos IS NOT NULL
GROUP BY estado
ORDER BY estado;

In [0]:
---Abandonment per stage
SELECT COUNT(CASE WHEN introduccion IS NOT NULL AND
                        solicitud_de_datos IS NULL AND
                        validacion_de_datos IS NULL AND
                        reconocimiento_facial IS NULL AND
                        firma IS NULL AND
                        confirmacion_de_datos IS NULL 
                  THEN 1 END) AS TI,
       COUNT(CASE WHEN  solicitud_de_datos IS NOT NULL AND
                        validacion_de_datos IS NULL AND
                        reconocimiento_facial IS NULL AND
                        firma IS NULL AND
                        confirmacion_de_datos IS NULL 
                  THEN 1 END) AS TSD,
       COUNT(CASE WHEN  validacion_de_datos IS NOT NULL AND
                        reconocimiento_facial IS NULL AND
                        firma IS NULL AND
                        confirmacion_de_datos IS NULL 
                  THEN 1 END) AS TVD,
       COUNT(CASE WHEN  reconocimiento_facial IS NOT NULL AND
                        firma IS NULL AND
                        confirmacion_de_datos IS NULL 
                  THEN 1 END) AS TRF,
       COUNT(CASE WHEN  firma IS NOT NULL AND
                        confirmacion_de_datos IS NULL 
                  THEN 1 END) AS TF
  FROM (SELECT * FROM digital_funnel.gold.pv_clientesfunnel WHERE proceso_completo = 0);

In [0]:
---Acarage per client to finish the process
SELECT AVG(DATEDIFF(confirmacion_de_datos, introduccion)) AS avg_finish_stages
  FROM digital_funnel.gold.pv_clientesfunnel
 WHERE proceso_completo = 1;

##Convertion per state - Funnel model

In [0]:
SELECT * FROM digital_funnel.gold.clientesfunnel;

In [0]:
-- Clients per state
SELECT estado, COUNT(*) AS total_clientes
FROM digital_funnel.gold.pv_clientesfunnel
GROUP BY estado
ORDER BY estado;

In [0]:
-- Total clients per stage (amount of clients that finished a stage)
SELECT COUNT(*) AS TI,
       COUNT(CASE WHEN solicitud_de_datos IS NOT NULL OR
                       validacion_de_datos IS NOT NULL OR
                       reconocimiento_facial IS NOT NULL OR
                       firma IS NOT NULL OR
                       confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TSD,
       COUNT(CASE WHEN  validacion_de_datos IS NOT NULL OR
                        reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TVD,
       COUNT(CASE WHEN  reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TRF,
       COUNT(CASE WHEN  firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TF,
       COUNT(CASE WHEN  confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TCD
  FROM (SELECT * FROM digital_funnel.gold.pv_clientesfunnel);

In [0]:
-- Conversion per state (stage, amount, conversion)

-- Define the amount of clients that finished a particlar stage 
WITH usuarios_por_estado AS (SELECT estado,
       COUNT(*) AS TI,
       COUNT(CASE WHEN solicitud_de_datos IS NOT NULL OR
                       validacion_de_datos IS NOT NULL OR
                       reconocimiento_facial IS NOT NULL OR
                       firma IS NOT NULL OR
                       confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TSD,
       COUNT(CASE WHEN  validacion_de_datos IS NOT NULL OR
                        reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TVD,
       COUNT(CASE WHEN  reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TRF,
       COUNT(CASE WHEN  firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TF,
       COUNT(CASE WHEN  confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TCD
  FROM digital_funnel.gold.pv_clientesfunnel
  WHERE estado IN (SELECT estado
                   FROM digital_funnel.gold.pv_clientesfunnel
                   GROUP BY estado)
  GROUP BY estado
),

-- Unpivot per stage
usuarios_por_etapa AS (
  SELECT estado, stage, total_usuarios
  FROM (
      SELECT estado, TI, TSD, TVD, TRF, TF, TCD
      FROM usuarios_por_estado
  ) AS src
  UNPIVOT (
      total_usuarios FOR stage IN (TI, TSD, TVD, TRF, TF, TCD)
  ) AS unpvt
)

-- Calculate the conversion rate
SELECT stage, 
       SUM(total_usuarios) AS total_by_stage,
       ROUND(SUM(total_usuarios) * 100.0 / 
             (SELECT SUM(total_usuarios) 
              FROM usuarios_por_etapa 
              WHERE stage = 'TI'), 2) AS porcentaje_conversion
FROM usuarios_por_etapa
GROUP BY stage
ORDER BY porcentaje_conversion DESC;

Databricks visualization. Run in Databricks to view.

In [0]:
-- Conversion per state (state, stage, amount, conversion)

-- Define the amount of clients that finished a particlar stage 
WITH usuarios_por_estado AS (SELECT estado,
       COUNT(*) AS TI,
       COUNT(CASE WHEN solicitud_de_datos IS NOT NULL OR
                       validacion_de_datos IS NOT NULL OR
                       reconocimiento_facial IS NOT NULL OR
                       firma IS NOT NULL OR
                       confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TSD,
       COUNT(CASE WHEN  validacion_de_datos IS NOT NULL OR
                        reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TVD,
       COUNT(CASE WHEN  reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TRF,
       COUNT(CASE WHEN  firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TF,
       COUNT(CASE WHEN  confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TCD
  FROM digital_funnel.gold.pv_clientesfunnel
  WHERE estado IN (SELECT estado
                   FROM digital_funnel.gold.pv_clientesfunnel
                   GROUP BY estado)
  GROUP BY estado
),

-- Unpivot per stage
usuarios_por_etapa AS (
  SELECT estado, stage, total_usuarios
  FROM (
      SELECT estado, TI, TSD, TVD, TRF, TF, TCD
      FROM usuarios_por_estado
  ) AS src
  UNPIVOT (
      total_usuarios FOR stage IN (TI, TSD, TVD, TRF, TF, TCD)
  ) AS unpvt
)

-- Calculate the conversion rate
SELECT a.estado, 
       a.stage, 
       a.total_usuarios,
       ROUND((a.total_usuarios * 100.0 / b.total_usuarios), 2) AS porcentaje_conversion
FROM usuarios_por_etapa AS a
JOIN usuarios_por_etapa AS b
ON (a.estado = b.estado AND b.stage = 'TI')
ORDER BY a.estado;

Databricks visualization. Run in Databricks to view.

In [0]:
WITH usuarios_por_estado AS (SELECT estado,
       CASE
         WHEN edad BETWEEN 18 AND 25 THEN '18–25'
         WHEN edad BETWEEN 26 AND 35 THEN '26–35'
         WHEN edad BETWEEN 36 AND 50 THEN '36–50'
         ELSE '50+'
       END AS edad_grupo,
       COUNT(*) AS TI,
       COUNT(CASE WHEN solicitud_de_datos IS NOT NULL OR
                       validacion_de_datos IS NOT NULL OR
                       reconocimiento_facial IS NOT NULL OR
                       firma IS NOT NULL OR
                       confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TSD,
       COUNT(CASE WHEN  validacion_de_datos IS NOT NULL OR
                        reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TVD,
       COUNT(CASE WHEN  reconocimiento_facial IS NOT NULL OR
                        firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TRF,
       COUNT(CASE WHEN  firma IS NOT NULL OR
                        confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TF,
       COUNT(CASE WHEN  confirmacion_de_datos IS NOT NULL 
                  THEN 1 END) AS TCD
  FROM digital_funnel.gold.pv_clientesfunnel
  WHERE estado IN (SELECT estado
                   FROM digital_funnel.gold.pv_clientesfunnel
                   GROUP BY estado)
  GROUP BY estado, edad_grupo
  ORDER BY estado, edad_grupo
),

usuarios_por_etapa AS (
  SELECT estado, edad_grupo, stage, total_usuarios
  FROM (
      SELECT estado, edad_grupo, TI, TSD, TVD, TRF, TF, TCD
      FROM usuarios_por_estado
  ) AS src
  UNPIVOT (
      total_usuarios FOR stage IN (TI, TSD, TVD, TRF, TF, TCD)
  ) AS unpvt
),

TI_per_edad_grupo AS (
  SELECT estado,
         stage,
         SUM(total_usuarios) OVER (PARTITION BY estado, stage) AS total_TI 
  FROM usuarios_por_etapa
  WHERE stage = 'TI'
  ORDER BY estado, stage, edad_grupo
)
SELECT a.estado,
       a.edad_grupo, 
       a.stage, 
       a.total_usuarios,
       ROUND((a.total_usuarios * 100.0 / b.total_TI), 2) AS porcentaje_conversion
FROM usuarios_por_etapa AS a
JOIN 
(SELECT estado, 
        MAX(total_TI) AS total_TI
FROM TI_per_edad_grupo
GROUP BY estado) AS b
ON (a.estado = b.estado); 


Databricks visualization. Run in Databricks to view.